# Xi0Stat Tutorial

## Setup

In [1]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats
from astropy.cosmology import FlatLambdaCDM

In [3]:
%cd ../

/Users/Michi/Dropbox/Local/Physics_projects/statistical_method_schutz_data_local/Xi0Stat


In [4]:
from Xi0Stat.globals import * 
from Xi0Stat.GW import get_all_events
from Xi0Stat.GLADE import GLADE
from Xi0Stat.GWENS import GWENS
from Xi0Stat.SYNTH import SYNTH
from Xi0Stat.completeness import *
from Xi0Stat.galCat import GalCompleted
from Xi0Stat.GWgal import GWgal
from Xi0Stat.betaHom import BetaHom

## GW

This module is based on the object Skymap3D. This contains the GW skymap for a single GW event denoted by its name - a string accessible by the attribute self.event_name. E.g.: 'GW170817'
Main attributes:
- npix: N of pixels
- nside : skymap nside
- pixarea: area of a pixel in rad^2
- p: probability in a pixel (the rho of Singer et al.)
- mu, sigma, norm: mu, sigma, N of the skymap 
- metadata: the metadata of the event. The metadata file should be a .csv file in the folder data/GW/metadata (editable in globals/metaPath). For O2, it should be called GWTC-1-confident.csv . The file in csv is downloadable at https://www.gw-openscience.org/eventapi/html/GWTC-1-confident/

Key methods to compute likelihood:

- likelihood(self, r, theta, phi) : likelihood given GW lum distance r, and polar angles theta, phi in radians 
- likelihood_px(self, r, pix) : lileihood  given GW lum distance r and pixel

(Note that the likelihood is a Gaussian truncated at zero. )

- likelihood_in_credible_region(self, r, level=0.99,) : likelihood in all the pixels of the credible region specified by level at given GW lum disance r
- d_max(self, SNR_ref=8): Max GW luminosity distance at which the evend could be seen, assuming its SNR and a threshold SNR_ref:d_max = d_obs*SNR/SNR_ref
- get_zlims(self, H0max=220, H0min=20, Xi0max=3, Xi0min=0.2) : Computes the possible range in redshift for the event given the prior range for H0 and XI0



Other: the module contains the function get_all_events(loc='data/GW/O2/', subset, subset_names) , that reads all the skymaps in the folder specified by loc and returns a dictionary {event name: Skymap3D}

If subset=True, the dictionary will contain only the events specified in subset_names, which should be a list of names, e.g. : get_all_events(loc='data/GW/O2/', subset=True, subset_names=['GW170817',]) to return skymap of GW170817

In [5]:
# Example: load skymaps of O2 events GW170817 

O2 = get_all_events(loc='data/GW/O2/', subset=True, subset_names=['GW170817',])

--- GW events:
['GW170817']
Reading skymaps....
Initializing Skymap3D...
Event name: GW170817 



In [6]:
# Example: chech area of the 90% credible region of GW170817

O2['GW170817'].area_p(pp=0.9)

16.15640267031496

## Completeness

In [7]:
compl = SkipCompleteness()

Initializing SkipCompleteness...


## GLADE

In [8]:
glade = GLADE('GLADE', compl, False)

Initializing GLADE...
Initializing GalCat...
Loading GLADE from /Users/Michi/Dropbox/Local/Physics_projects/statistical_method_schutz_data_local/Xi0Stat/Xi0Stat/../data/GLADE/GLADE_2.4.txt...
N. of objects: 3263611
Dropping galaxies with HyperLeda name=null and flag2=2...
Kept 3262083 points or 100% of total
Computing cosmological redshifts from given luminosity distance with H0=70.0 km / (Mpc s), Om0=0.27...
Interpolating between z_min=0, z_max=6.49511599474
2720966 points have valid entry for dist
541117 points have null entry for dist, correcting original redshift
Loading galaxy group catalogue from /Users/Michi/Dropbox/Local/Physics_projects/statistical_method_schutz_data_local/Xi0Stat/Xi0Stat/../data/misc/Galaxy_Group_Catalogue.csv...
Correcting z_cosmo for group velocities...
Correcting z_cosmo for CMB reference frame...
Keeping only galaxies with positive redshift in the colums z_cosmo_CMB...
Kept 2720852 points or 83% of total
Renaming column z_cosmo_CMB to z. This will be used

In [9]:
gals = GalCompleted()
gals.add_cat(glade)

Initializing GalCompleted...


## GWgal

This module contains the object GWgal. Its main attributes are :
- gals : a GalCompleted object
- GWevents: a dictionary {event name: Skymap3D} for the GW events

At initialization, two other attributes will be created:
- cred_pixels : a dictionary {event name: list of pixels of the credible region specified by credible_level (default 0.99)}
- z_lims : a dictionary {event name: (z_min, z_max) } where z_min, z_max are the min and max redshift for all prior ranges in H0 and Xi0 . This is computed using mean+-3sigma with mean and sigma the mu and std of the GW luminosity distance, taken from the header of the skymap 

In [10]:
myGWgal = GWgal(gals, O2)


 --- GW events: 
GW170817


In [11]:
# Example: z limits for GW170817 with prior range H0max=220, H0min=20, Xi0max=3, Xi0min=0.2

myGWgal.z_lims

{'GW170817': (0.0010314992453354975, 0.04593987289261164)}

## Computing posterior

In [12]:
# Set area and z range in the galaxy catalogue

myGWgal.gals.set_z_range( *myGWgal.z_lims['GW170817'])
myGWgal.gals.set_area(myGWgal.cred_pixels['GW170817'], myGWgal.GWevents['GW170817'].nside)

Setting z range of the catalogue between 0.0010314992453354975, 0.04593987289261164
241118 galaxies kept
Restricting area of the catalogue to 4928 pixels with nside=1024
1020 galaxies kept


In [13]:
# Compute likelihood for H0 on a grid
H0grid = np.linspace(20, 220, 30)
lik = np.array([myGWgal.get_inhom_lik(H0, Xi0=1, n=1.91) for H0 in H0grid])

TypeError: ufunc 'true_divide' output (typecode 'O') could not be coerced to provided output parameter (typecode 'd') according to the casting rule ''same_kind''

In [ ]:
# Plot the posterior with beta=H0^3

norm=np.trapz(lik/H0grid**3, H0grid)

plt.plot(H0grid, lik/H0grid**3/norm)